<h1 style="text-align: center">Customising nbconvert</h1>

<h3 style="text-align: center">How to turn Jupyter notebooks into anything you want</h3>

<p style="font-weight:bold; text-align: center">Thomas Kluyver & Min Ragan-Kelley</p>

<p style="text-align: center">PyData London 2016</p>

## Introducing nbconvert

![nbconvert flow chart](nbconvert.png)

## Nbconvert at the command line

With files:

    jupyter nbconvert --to slides Presentation.ipynb
    # This creates Presentation.html

With pipes:

    ... | jupyter nbconvert --stdin --execute --to notebook --stdout | ...

## Nbconvert as a Python API

In [22]:
import nbformat
from nbconvert.exporters import RSTExporter

nb = nbformat.read('Presentation.ipynb', as_version=4)
rst, resources = RSTExporter().from_notebook_node(nb)
slide_start = rst.find('Nbconvert as a Python API')
print(rst[slide_start:slide_start+200])

Nbconvert as a Python API
-------------------------

.. code:: python

    import nbformat
    from nbconvert.exporters import MarkdownExporter
    
    nb = nbformat.read('Presentation.ipynb', as_ver


nbparameterise demo

## Nbconvert as a web service

In [21]:
from IPython.display import IFrame
IFrame('http://nbviewer.jupyter.org/url/norvig.com/ipython/Countdown.ipynb', width=800, height=600)

## Custom templates

In [38]:
IFrame('http://nbconvert.readthedocs.io/en/latest/customizing.html#Template-structure', width=750, height=600)